In [ ]:
from datasets import load_dataset, Dataset
import json
from tqdm import tqdm
GPT_API_KEY = "sk-XXX"
import openai
client = openai.OpenAI(api_key=GPT_API_KEY)

/home/infidea/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
origin_data = load_dataset("mncai/finance-tasks-ConvFinQA-ko")['train']

In [24]:
def make_cot(data):
    res_list = []
    PROMPT = """ 
    당신은 금융, 재무 전문가입니다.
    주어진 문제를 보고 풀이와 답을 알려주세요.
    
    요구사항: 
    - 문제에서 제공하는 정보를 파악하고 생각의 과정을 각각 한 문장으로 작성하세요.
    - '따라서 정답은 XXX입니다' 로 풀이를 마무리 해주세요.
    - 파악한 정보를 토대로 풀이를 작성하되 정보를 다시 작성하지는 마세요.
    문제:
    {question}
    """
    question = data['input']
    prompt = PROMPT.format(question=question)
    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
    ],
    )
    response_txt  = response.choices[0].message.content
    return response_txt

In [ ]:
res_list = []
for datum in tqdm(origin_data):
    answer = make_cot(datum)
    res_dict = {'question': datum['input'],
                'answer': answer}
    res_list.append(res_dict)

100%|██████████| 1208/1208 [1:20:52<00:00,  4.02s/it]


In [35]:
for l in res_list:
    l['answer'] = l['answerr']

In [38]:
res_list = [{k: v for k, v in d.items() if k != 'answerr'} for d in res_list]

In [39]:
res_list[0]

{'question': '2017년 12월 31일 현재, 이 회사는 2018년부터 2020년까지 만료되는 약 2,000만 달러의 총 주 소득세 세액 공제 이월 forwards을 보유하고 있다. 이러한 주 소득세 세액 공제 이월 forwards과 관련하여 약 1,600만 달러(연방 수혜 순)의 이연법인세 자산이 구축되어 있다, 2017년 12월 31일 현재 이러한 이연법인세자산에 대한 평가충당금은 700만 달러이다. 이 회사의 총 상태순영업손실 이월액은 3,900만 달러로 2027년에 만기가 도래한다. 순영업손실 이월액에 대해 약 300만 달러(연방급여순액)의 이연법인세자산이 설정되어 있다, 2017년 12월 31일 기준으로 전액 평가 수당을 지급합니다. 기타 국가 및 외국인 순영업손실 이월- forwards는 2019년 이연법인세 잔액에 별도로 누적적으로 중요하지 않으며 2026년에서 2036년 사이에 만기가 도래한다. 14. 부채 장기부채는 다음과 같이 구성되었다.\n\n($-million) | 2017년 12월 31일 | 2016년 12월 31일\n2021년 12월 15일 만기 선순위 채권 5.000% (5.000%) | 2014 | 600\n2025년 11월 15일 만기 선순위 채권 5.000% (5.000%) | 600 | 600\n2027년 12월 12일 만기 선순위 채권 3.483% (3.483%) | 600 | 2014\n미시시피 경제 개발 수익 채권은 2024년 5월 1일까지 7.81%(7.81%) | 84 | 84\n걸프오퍼튜니티존 12월 12일 만기 산업개발수익채권 4.55% (4.55%) | 21 | 21\n분할되지 않은 부채 발행 비용 감소 | -26(26) | -27(27)\n총 장기 부채 | 1279 | 1278\n\n신용 시설 - 2017년 11월, 회사는 두 번째 수정 및 재작성된 신용 계약을 해지하고 제3자 대출 기관과 새로운 신용 계약("신용 시설")을 체결했다. 리볼빙 신용시설은 2017년 11월 22일부터 5년간 차입 가능한

In [41]:
len(res_list)

1208

In [40]:
with open('convfinqa_cot.json','w',encoding='utf-8') as f:
    json.dump(res_list,f,ensure_ascii=False, indent=4)


res_data =Dataset.from_list(res_list)
res_data.push_to_hub("overfit-brothers/ConvFinQA-ko-cot")


Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.11s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/overfit-brothers/ConvFinQA-ko-cot/commit/192f1fba113cfa3553d13e6a02dc7e2cc5e60b1c', commit_message='Upload dataset', commit_description='', oid='192f1fba113cfa3553d13e6a02dc7e2cc5e60b1c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/overfit-brothers/ConvFinQA-ko-cot', endpoint='https://huggingface.co', repo_type='dataset', repo_id='overfit-brothers/ConvFinQA-ko-cot'), pr_revision=None, pr_num=None)